In [131]:
import pandas as pd
import spotipy  
from spotipy.oauth2 import SpotifyClientCredentials
import os
from dotenv import load_dotenv
from musixmatch import Musixmatch
import nltk
from nltk.corpus import stopwords
import random

CLIENT_ID = os.getenv('CLIENT_ID')
SECRET_ID = os.getenv('SECRET_ID')
ACCESS_TOKEN = os.getenv('ACCESS_TOKEN')
MUSICXMATCH = os.getenv('MUSICXMATCH')



musixmatch = Musixmatch(MUSICXMATCH)

client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=SECRET_ID)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


# load environment variables from .env file
load_dotenv()




palabras = stopwords.words('spanish')
#I suspect that the word 'no' is a very important word when we are talking about love or hate
#so I will remove it from the stopwords list
palabras.remove('no')




In [167]:


class Song:
    def __init__(self, name, artist, label, genre = None):
        self.name = name
        self.artist = artist
        self.genre = genre
        self.label = label

    def __str__(self):
        return self.name + " by " + self.artist


    def get_lyrics(self,):
        song = musixmatch.matcher_track_get(self.name,self.artist)
        track_id = song['message']['body']['track']['track_id']
        lyrics = musixmatch.track_lyrics_get(track_id=track_id)

        if lyrics['message']['body'] == []:
            pass
        else:
            lyrics = lyrics['message']['body']['lyrics']['lyrics_body'].split("...")[0]
        return lyrics


    def get_name(self):
        return self.name

    def get_artist(self):
        return self.artist

    def get_words(self):
       words = nltk.word_tokenize(self.get_lyrics())
       words = [word.lower() for word in words if word.isalpha()]
       words = [word for word in words if word not in palabras]
       return words
    
    def get_label(self):
        return self.label
    

    def get_id(self):
        results = sp.search(q='track:' + self.name + ' artist:' + self.artist, type='track')
        return results['tracks']['items'][0]['id']
        
    



class Playlist:

    all_playlists = []

    def __init__(self,url,label, genre=None):
        self.url = url
        self.genre = genre
        self.label = label
        self.__class__.all_playlists.append(self)
  

    #results.keys()
    #dict_keys(['public', 'snapshot_id', 'tracks', 'type', 'uri'])

    #results["tracks"].keys()
    #dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])


    #Items is a list of tracks 
    def get_songs(self):
        results = sp.playlist(self.url)["tracks"]
        tracks = results['items']
        while results['next']:
            results = sp.next(results)
            tracks.extend(results['items'])
    
        SongList = []
        for track in tracks:
            SongList.append(Song(track['track']['name'], track['track']['artists'][0]['name'],self.label,genre=self.genre))

        return SongList

    
    def info(self):
        print(" Genre: " +self.genre,"- Size: " + str(len(self.get_songs())),"- Label: ",self.label)

    @classmethod
    def total_length(cls):
        total = 0
        for playlist in cls.all_playlists:
            total += len(playlist.get_songs())
        return total



ranch_1 = Playlist('https://open.spotify.com/playlist/1oj5pzkKVfNDg6MDhk2XHf?si=b279724e3aaf45ab',1,'ranchera')
regga_1 = Playlist('https://open.spotify.com/playlist/5d2djDHRUTDOgTuTjDGpk5',1,'reggaeton')
mixed_1 = Playlist('https://open.spotify.com/playlist/0nCQsteTsCEno08krzKai6',1,'mixed')
mixed_2 = Playlist('https://open.spotify.com/playlist/4bPJZRZ3K7mVqoOoHQDipu',1,'mixed')
mixed_3 = Playlist('https://open.spotify.com/playlist/7fnKiS1dem6SlOhl9DM1Bd',1,'mixed')
regga_3 = Playlist('https://open.spotify.com/playlist/1wylHL6nSxcYHlGf5mDzR2',1,'reggaeton')
ranch_2 = Playlist('https://open.spotify.com/playlist/1mwDUXYSjQWKnbvWvu11TP',1,'ranchera')
ranch_3 = Playlist('https://open.spotify.com/playlist/7DhZIOgM52v1LHOly8icGU',1,'ranchera')
bacha_1 = Playlist('https://open.spotify.com/playlist/1hTrF2j2ckMGrjM5myfBBE',1,'bachata')



ranch_1.info()
regga_1.info()
mixed_1.info()
mixed_2.info()
mixed_3.info()
regga_3.info()
ranch_2.info()
ranch_3.info()
bacha_1.info()



Playlist.total_length()




 Genre: ranchera - Size: 84 - Label:  1
 Genre: reggaeton - Size: 109 - Label:  1
 Genre: mixed - Size: 194 - Label:  1
 Genre: mixed - Size: 810 - Label:  1
 Genre: mixed - Size: 127 - Label:  1
 Genre: reggaeton - Size: 204 - Label:  1
 Genre: ranchera - Size: 252 - Label:  1
 Genre: ranchera - Size: 352 - Label:  1
 Genre: bachata - Size: 124 - Label:  1


2256